## 크롤링 crawling

##### 홈페이지 앞에 view-source:를 붙이면 홈페이지의 소스 확인 가능
##### 일반화:
##### <여는태그>내용</닫는태그>
##### <여는태그 속성명="속성값">내용</닫는태그>
##### <여는태그 속성명="속성값"/>
##### html부터 /html까지 하나의 덩어리, head부터 /head까지 하나의 덩어리, body부터 /body까지 하나의 덩어리
##### l는 end를 의미함

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

##### import A는 A를 메모리로 불러옴
##### 상위폴더 안에 있는 하위폴더를 한번에 불러오고 싶으면 from 상위폴더 import 하위폴더의 형태로 쓰면 됨

In [ ]:
html = urlopen("http://www.naver.com")
html

##### urlopen을 통해 받아온 네이버를 BeautifulSoup을 통해 실행하면 view-sorce:https://naver.com의 내용을 받아올 수 있음

## 웹문서 전체를 가져오기

In [ ]:
bsObject = BeautifulSoup(html, "html.parser")
bsObject.html

##### bsObject.html은 html 부분의 내용을 받음. 이 속에서 head, title 부분을 가져올 수도 있음

## head 가져오기

In [ ]:
bsObject.head

##### head 부분에서 8번째 줄은 /가 없기 때문에 한줄로 끝나지 않음. 맨 마지막 줄의 마지막 /meta에서 끝이 남

## title 가져오기 -> 3/19 시작!

In [ ]:
bsObject.head.title

In [ ]:
bsObject.head.title.get_text()  # bsObject.head.title.text.strip()